In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import gc
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
import warnings

warnings.filterwarnings('ignore')


In [2]:
%run utils.py
%run data_process.py

===========================single one===============================
                   

In [116]:
train_data_dir = '/home/hou/Windy/IJCAI18/fusai/data/sub0.txt'
test_data_dir_1= '/home/hou/Windy/IJCAI18/data/round1_ijcai_18_test_a_20180301.txt'
test_data_dir_2 = '/home/hou/Windy/IJCAI18/fusai/data/test.txt'
test_save = pd.read_csv(test_data_dir_2,sep=" ")
test_save['model']=test_save['user_id'].apply(lambda x:x%10)
test_save['predicted_score']=0


In [3]:
def get_peng_feature(data):
    print('peng_feature')
    data['context_timestamp_str'] = data['context_timestamp'].astype(str)
    user_time_join = data.groupby('user_id')['context_timestamp_str'].agg(lambda x:';'.join(x)).reset_index()
    user_time_join.rename(columns={'context_timestamp_str':'user_time_join'},inplace = True)
    data = pd.merge(data,user_time_join,on=['user_id'],how='left')
    user_shop_time_join = data.groupby(['user_id','shop_id'])['context_timestamp_str'].agg(lambda x:';'.join(x)).reset_index()
    user_shop_time_join.rename(columns={'context_timestamp_str':'user_shop_time_join'},inplace = True)
    data = pd.merge(data,user_shop_time_join,on=['user_id','shop_id'],how='left')
    user_item_time_join = data.groupby(['user_id','item_id'])['context_timestamp_str'].agg(lambda x:';'.join(x)).reset_index()
    user_item_time_join.rename(columns={'context_timestamp_str':'user_item_time_join'},inplace = True)
    data = pd.merge(data,user_item_time_join,on=['user_id','item_id'],how='left')
    data['index_']=data.index
    del user_time_join,user_shop_time_join,user_item_time_join

    def get_20min_user_count(index_):
        num=0
        df1 = data.context_timestamp[index_]
        df2 = data.user_time_join[index_].split(';')
        df2_len = len(df2)
        for i in range(df2_len):
            if(((int(df2[i])-df1)<600)&((int(df2[i])-df1)>-600)):
                num+=1
        return num
    def get_20min_user_shop_count(index_):
        num=0
        df1 = data.context_timestamp[index_]
        df2 = data.user_shop_time_join[index_].split(';')
        df2_len = len(df2)
        for i in range(df2_len):
            if(((int(df2[i])-df1)<600)&((int(df2[i])-df1)>-600)):
                num+=1
        return num
    def get_20min_user_item_count(index_):
        num=0
        df1 = data.context_timestamp[index_]
        df2 = data.user_item_time_join[index_].split(';')
        df2_len = len(df2)
        for i in range(df2_len):
            if(((int(df2[i])-df1)<600)&((int(df2[i])-df1)>-600)):
                num+=1
        return num
    a=time.time()
    data['user_count_20']=data.index_.apply(get_20min_user_count)
    print time.time()-a

    a=time.time()
    data['user_shop_count_20']=data.index_.apply(get_20min_user_shop_count)
    print time.time()-a

    a=time.time()
    data['user_item_count_20']=data.index_.apply(get_20min_user_item_count)
    print time.time()-a
    return data


In [4]:
#user item last one / all times
#user category last one / all times
def get_last_feature(data):
    tt=time.time()
    item_last = data[['user_id','item_id','day','context_timestamp_str']]
    item_last = item_last.groupby(['day','user_id','item_id'])['context_timestamp_str'].agg(lambda x:';'.join(x)).reset_index()
    item_last['user_item_day_num']=item_last.context_timestamp_str.apply(lambda s:len(s.split(';')))
    data = pd.merge(data,item_last[['user_id','item_id','day','user_item_day_num']],on=['user_id','item_id','day'],how='left')
   # print(data.shape)
    item_last1 =item_last[item_last.user_item_day_num>1] 
   # print(item_last1.shape)
    item_last1['max_time']=item_last1.context_timestamp_str.apply(lambda s:max([int(d) for d in s.split(';')]))
   # print(item_last1.shape)
    item_last1=item_last1[['user_id','item_id','day','max_time']]
   # print(item_last1.shape)
   # item_last = data.copy()
   # print(data.shape)
    data = pd.merge(data,item_last1,on=['user_id','item_id','day'],how='left')
   # print(data.shape)
    data['u_i_d_last_time']=data.max_time-data.context_timestamp_str.astype(int)
   # print(data.shape)
    def is_firstlastone(x):
        if x==0:
            return 1
        elif x>0:
            return 0
        else :
            return -1 #only receive once
    data.u_i_d_last_time = data.u_i_d_last_time.apply(is_firstlastone)
    #print(data.shape)
    #data = pd.merge(data,item_last[['user_id','item_id','day','u_i_d_last_time']],on=['user_id','item_id','day'],how='left')
    #print(data.shape)
    cate_last = data[['user_id','cate_id','day','context_timestamp_str']]
    cate_last = cate_last.groupby(['day','user_id','cate_id'])['context_timestamp_str'].agg(lambda x:';'.join(x)).reset_index()
    cate_last['user_cate_day_num'] = cate_last.context_timestamp_str.apply(lambda s:len(s.split(';')))
    data= pd.merge(data,cate_last[['user_id','cate_id','day','user_cate_day_num']],on=['user_id','cate_id','day'],how='left')
    #print(data.shape)
    cate_last1 = cate_last[cate_last.user_cate_day_num>1]
    cate_last1['c_max_time']=cate_last1.context_timestamp_str.apply(lambda s:max([int(d) for d in s.split(';')]))
    cate_last1=cate_last1[['user_id','cate_id','day','c_max_time']]
    
    #cate_last = data[['user_id','cate_id','day','context_timestamp_str']]
    data = pd.merge(data,cate_last1,on=['user_id','cate_id','day'],how='left')
    data['u_c_d_last_time'] = data.c_max_time-data.context_timestamp_str.astype(int)
    data.u_c_d_last_time = data.u_c_d_last_time.apply(is_firstlastone)
  #  data = pd.merge(data,cate_last[['user_id','cate_id','day','u_c_d_last_time']],on=['user_id','cate_id','day'],how='left')
    print(data.shape)
    print('get_last_time: ',time.time()-tt)

    return data








In [117]:
print 'start'
singletime=time.time()
data=pd.read_csv(train_data_dir,sep=" ")
data=data.append(test_save[test_save.model==0]).reset_index().drop(['index'],axis=1)
data['is_trade']=data['is_trade'].fillna(0)
data = data.sort_values(by='context_timestamp').reset_index().drop(['index'],axis=1)
gc.collect()
print(data.shape)
data=data_process0(data)
print(data.shape)
data=data_process1(data)
print(data.shape)
data=data_process2(data)
print(data.shape)
data=data_process3(data)
print(data.shape)
data=data_process4(data)
print(data.shape)
data=data_process6(data)#make some change from data_process5
print(data.shape)
data=get_peng_feature(data)
print(data.shape)
data=get_last_feature(data)
print(data.shape)

rank_list=['user_price_rank','user_sale_rank','user_collect_rank','pre_cate_price_rank','pre_cate_sale_rank','pre_cate_collect_rank']
for col in rank_list:
    if '%s_maxrank'%col in data.columns:
        del data['%s_maxrank'%col]
    tmp=data.groupby('user_day',as_index=False)[col].max().rename(columns={col:'%s_maxrank'%col})
    data=data.merge(tmp,on='user_day',how='left')
    data['%s_maxrank'%col]=data['%s_maxrank'%col].astype(float)
    data[col]=data[col]/data['%s_maxrank'%col]
    del data['%s_maxrank'%col]
    
day={4:data[data.day==4].copy(),5:data[data.day==5].copy(),
    6:data[data.day==6].copy(),7:data[data.day==7].copy()}
    
count_list=[]
for cols in [['shop_id'],['item_id'],['user_id'],['item_brand_id'],
            ['item_city_id'],['user_id','item_id'],['user_id','shop_id'],['user_id','item_brand_id'],['user_id','item_city_id']]: 
    for day_v in xrange(4,8):#0,1,2,3is use for generate count
        if len(cols)==1:
            name='cnt_'+cols[0]
        else:
            name='cnt_'+cols[0]+'_'+cols[1]
        day[day_v]=day[day_v].merge(get_silde_cnt(data,day_v,cols,name),on=cols,how='left')
        day[day_v][name]=day[day_v][name].fillna(0)
        gc.collect()
    count_list+=[name]
print count_list

for day_v in xrange(4,8):
    day[day_v]['item_user_ratio']=day[day_v]['cnt_user_id_item_id']/day[day_v]['cnt_user_id']
    day[day_v]=get_count(day[day_v],['user_id'], 'cnt_user_tmp', 'is_trade')
    day[day_v]=get_count(day[day_v],['user_id','item_id'], 'cnt_user_item_tmp', 'is_trade')
    day[day_v]['item_user_ratio2']=day[day_v]['cnt_user_item_tmp']/day[day_v]['cnt_user_tmp']
    del day[day_v]['cnt_user_tmp']
gc.collect()
print time.time()-singletime

start
(1094123, 29)
encode cate id now
(1094123, 32)
fill nan now
(1094123, 32)
encode now
(1094123, 32)
decode time
(1094123, 34)
cate_match
(1094123, 40)
tongji2
(1094123, 53)
peng_feature
64.4940881729
53.6697490215
65.9731240273
(1094123, 61)
(1094123, 67)
('get_last_time: ', 43.05160999298096)
(1094123, 67)
['cnt_shop_id', 'cnt_item_id', 'cnt_user_id', 'cnt_item_brand_id', 'cnt_item_city_id', 'cnt_user_id_item_id', 'cnt_user_id_shop_id', 'cnt_user_id_item_brand_id', 'cnt_user_id_item_city_id']
431.570374966


In [51]:
filter_day7=day[7].user_id%2==0
train=pd.concat([day[4],day[5],day[6],day[7][(day[7].model.isnull())&(filter_day7)]]).copy().reset_index().drop('index',axis=1)
val=day[7][(day[7].model.isnull())&(~filter_day7)].copy().reset_index().drop('index',axis=1)
test=day[7][day[7].model.notnull()].copy().reset_index().drop('index',axis=1)

(1098983, 70)

In [ ]:

canshu=1
train['weight']=1
train['weight'][train.day==7]=canshu
train_weight=train['weight'].values
train_weight_list = np.array(train_weight).tolist()

val['weight']=1
val['weight'][val.day==7]=canshu
val_weight=val['weight'].values
val_weight_list = np.array(val_weight).tolist()
singletime=time.time()

last_feature=['u_i_d_last_time','user_item_day_num','user_cate_day_num','u_c_d_last_time']
peng_feature=['user_count_20','user_shop_count_20','user_item_count_20']
cate_feature=['user_gender_id','user_occupation_id','cate_id','day']
features=[ 'item_price_level', 'item_sales_level','item_collected_level','item_pv_level','user_age_level', 'user_star_level',
         'shop_review_num_level', 'shop_star_level','shop_review_positive_rate', 'shop_score_service','shop_score_delivery', 
        'shop_score_description','cate_match','context_page_id', 'hour']+['item_user_ratio','item_user_ratio2','cnt_user_item_tmp',
                  'pre_cate_price_rank','pre_cate_sale_rank','pre_cate_collect_rank','user_shop_cumcount',
         'user_item_cumcount','shop_item_nunique','user_price_rank','user_sale_rank','user_collect_rank'                                                        
         ]+\
        cate_feature+count_list
    #+peng_feature+last_feature
target='is_trade'



gbm = lgb.LGBMClassifier(objective='binary',
                                max_depth=5,
                                num_leaves=16,
                                learning_rate=0.02,
                                n_estimators=2000,
                                colsample_bytree=0.3,#0.3
                                subsample = 0.75,#0.75
                                n_jobs=4,
                                lambda_l2=10,
                                seed=2018,

                                )
gbm.fit(train[features],train[target],sample_weight=train_weight_list,categorical_feature=cate_feature,
                    eval_set=[(val[features],val[target])],eval_sample_weight=None,
                    eval_metric='binary_logloss',
                    early_stopping_rounds=40,verbose=0)




val_error=gbm.best_score_['valid_0']['binary_logloss']
best_model=gbm.booster_
train_pred=best_model.predict(train[features], num_iteration=best_model.best_iteration)
print val_error,log_loss(train[target],train_pred)
print time.time()-singletime

In [11]:
features

['item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level',
 'user_age_level',
 'user_star_level',
 'shop_review_num_level',
 'shop_star_level',
 'shop_review_positive_rate',
 'shop_score_service',
 'shop_score_delivery',
 'shop_score_description',
 'cate_match',
 'context_page_id',
 'hour',
 'item_user_ratio',
 'item_user_ratio2',
 'cnt_user_item_tmp',
 'pre_cate_price_rank',
 'pre_cate_sale_rank',
 'pre_cate_collect_rank',
 'user_shop_cumcount',
 'user_item_cumcount',
 'shop_item_nunique',
 'user_price_rank',
 'user_sale_rank',
 'user_collect_rank',
 'user_gender_id',
 'user_occupation_id',
 'cate_id',
 'day',
 'cnt_shop_id',
 'cnt_item_id',
 'cnt_user_id',
 'cnt_item_brand_id',
 'cnt_item_city_id',
 'cnt_user_id_item_id',
 'cnt_user_id_shop_id',
 'cnt_user_id_item_brand_id',
 'cnt_user_id_item_city_id',
 'user_count_20',
 'user_shop_count_20',
 'user_item_count_20']

=======================================all=================================================
                                      

In [5]:
def get_rate_feature(day7,data):
    print(data.shape,day7.shape)
    data=data[data.day!=7]
    print(data.shape)
    data2=data[['item_id','is_trade']]
    data2['item_trade_all']=1
    data2['item_trade_yes']=1
    data2_all = data2.groupby(['item_id'],as_index=False).agg({'item_trade_all':'count'})
    data2_yes = data2[data2.is_trade==1].groupby(['item_id'],as_index=False).agg({'item_trade_yes':'count'})
    data2 = pd.merge(data2_all,data2_yes,on=['item_id'],how='left')
    del data2_all,data2_yes
    data2 = data2.fillna(0)
    data2['item_rate']=(data2.item_trade_yes+0.001)/(data2.item_trade_all+0.001)
    data2=data2[['item_id','item_rate','item_trade_all','item_trade_yes']]
    day7 = pd.merge(day7,data2,on=['item_id'],how='left')
    data3=data[['user_id','is_trade']]
    data3['user_trade_all']=1
    data3['user_trade_yes']=1
    data3_all = data3.groupby(['user_id'],as_index=False).agg({'user_trade_all':'count'})
    data3_yes = data3[data3.is_trade==1].groupby(['user_id'],as_index=False).agg({'user_trade_yes':'count'})
    data3 = pd.merge(data3_all,data3_yes,on=['user_id'],how='left')
    del data3_all,data3_yes
    data3 = data3.fillna(0)
    data3['user_rate']=(data3.user_trade_yes+0.001)/(data3.user_trade_all+0.001)
    data3=data3[['user_id','user_rate','user_trade_all','user_trade_yes']]
    day7 = pd.merge(day7,data3,on=['user_id'],how='left')
    return day7
    

In [16]:
def get_rate_feature_large(day7,data):
    print(data.shape,day7.shape)
    data=data[data.day!=7]
    print(data.shape)
    data2=data[['item_id','is_trade']]
    data2['l_item_trade_all']=1
    data2['l_item_trade_yes']=1
    data2_all = data2.groupby(['item_id'],as_index=False).agg({'l_item_trade_all':'count'})
    data2_yes = data2[data2.is_trade==1].groupby(['item_id'],as_index=False).agg({'l_item_trade_yes':'count'})
    data2 = pd.merge(data2_all,data2_yes,on=['item_id'],how='left')
    del data2_all,data2_yes
    data2 = data2.fillna(0)
    data2['l_item_rate']=(data2.l_item_trade_yes+0.001)/(data2.l_item_trade_all+0.001)
    data2=data2[['item_id','l_item_rate','l_item_trade_all','l_item_trade_yes']]
    day7 = pd.merge(day7,data2,on=['item_id'],how='left')
    data3=data[['user_id','is_trade']]
    data3['l_user_trade_all']=1
    data3['l_user_trade_yes']=1
    data3_all = data3.groupby(['user_id'],as_index=False).agg({'l_user_trade_all':'count'})
    data3_yes = data3[data3.is_trade==1].groupby(['user_id'],as_index=False).agg({'l_user_trade_yes':'count'})
    data3 = pd.merge(data3_all,data3_yes,on=['user_id'],how='left')
    del data3_all,data3_yes
    data3 = data3.fillna(0)
    data3['l_user_rate']=(data3.l_user_trade_yes+0.001)/(data3.l_user_trade_all+0.001)
    data3=data3[['user_id','l_user_rate','l_user_trade_all','l_user_trade_yes']]
    day7 = pd.merge(day7,data3,on=['user_id'],how='left')
    print day7.shape
    return day7

In [12]:
timm=time.time()
test_save=pd.read_csv('/home/hou/Windy/IJCAI18/fusai/data/test.txt',sep=" ")
test_save['model']=test_save['user_id'].apply(lambda x:x%10)
test_save['predicted_score']=0

for i in xrange(10):
    print
    print "data",i

    data=pd.read_csv('/home/hou/Windy/IJCAI18/fusai/data/sub%d.txt'%i,sep=" ")
    data=data.append(test_save[test_save.model==i]).reset_index().drop(['index'],axis=1)
    data['is_trade']=data['is_trade'].fillna(0)
    data = data.sort_values(by='context_timestamp').reset_index().drop(['index'],axis=1)
    gc.collect()
    print(data.shape)
    data=data_process0(data)
    print(data.shape)
    data=data_process1(data)
    print(data.shape)
    data=data_process2(data)
    print(data.shape)
    data=data_process3(data)
    print(data.shape)
    data=data_process4(data)
    print(data.shape)
    data=data_process6(data)#make some change from data_process5
    print(data.shape)
    data=get_peng_feature(data)
    print(data.shape)
    data=get_last_feature(data)

    rank_list=['user_price_rank','user_sale_rank','user_collect_rank','pre_cate_price_rank','pre_cate_sale_rank','pre_cate_collect_rank']
    for col in rank_list:
        if '%s_maxrank'%col in data.columns:
            del data['%s_maxrank'%col]
        tmp=data.groupby('user_day',as_index=False)[col].max().rename(columns={col:'%s_maxrank'%col})
        data=data.merge(tmp,on='user_day',how='left')
        data['%s_maxrank'%col]=data['%s_maxrank'%col].astype(float)
        data[col]=data[col]/data['%s_maxrank'%col]
        del data['%s_maxrank'%col]

        
        

    day={4:data[data.day==4].copy(),5:data[data.day==5].copy(),
        6:data[data.day==6].copy(),7:data[data.day==7].copy()}

    
    
    count_list=[]
    for cols in [['shop_id'],['item_id'],['user_id'],['item_brand_id'],
                ['item_city_id'],['user_id','item_id'],['user_id','shop_id'],['user_id','item_brand_id'],['user_id','item_city_id']]: 
        for day_v in xrange(4,8):#0,1,2,3is use for generate count
            if len(cols)==1:
                name='cnt_'+cols[0]
            else:
                name='cnt_'+cols[0]+'_'+cols[1]
            day[day_v]=day[day_v].merge(get_silde_cnt(data,day_v,cols,name),on=cols,how='left')
            day[day_v][name]=day[day_v][name].fillna(0)
            gc.collect()
        count_list+=[name]
    print count_list

    for day_v in xrange(4,8):
        day[day_v]['item_user_ratio']=day[day_v]['cnt_user_id_item_id']/day[day_v]['cnt_user_id']
        day[day_v]=get_count(day[day_v],['user_id'], 'cnt_user_tmp', 'is_trade')
        day[day_v]=get_count(day[day_v],['user_id','item_id'], 'cnt_user_item_tmp', 'is_trade')
        day[day_v]['item_user_ratio2']=day[day_v]['cnt_user_item_tmp']/day[day_v]['cnt_user_tmp']
        del day[day_v]['cnt_user_tmp']
    print(day[7].shape)
    day[7]=get_rate_feature(day[7],data)
    print(day[7].shape)
    rate_save=data[['user_id','item_id','day','is_trade']]
    if(i==0):
        allday7=pd.DataFrame(columns=day[7].columns)
    if(i==0):
        rate_saveall=pd.DataFrame(columns=rate_save.columns)
    allday7=pd.concat([allday7,day[7]])
    rate_saveall=pd.concat([rate_saveall,rate_save])
    print(day[7].shape,allday7.shape)
    print(rate_saveall.shape,rate_save.shape)
print(allday7.shape,rate_save.shape)
allday7.to_csv('/home/hou/Windy/IJCAI18/fusai/data/day7rate.csv',index=False)
rate_saveall.to_csv('/home/hou/Windy/IJCAI18/fusai/data/rate_saveall.csv',index=False)
print time.time()-timm


data 0
(1094123, 29)
encode cate id now
(1094123, 32)
fill nan now
(1094123, 32)
encode now
(1094123, 32)
decode time
(1094123, 34)
cate_match
(1094123, 40)
tongji2
(1094123, 53)
peng_feature
64.4258580208
49.5462400913
48.9799849987
(1094123, 61)
(1094123, 67)
('get_last_time: ', 40.768515825271606)
['cnt_shop_id', 'cnt_item_id', 'cnt_user_id', 'cnt_item_brand_id', 'cnt_item_city_id', 'cnt_user_id_item_id', 'cnt_user_id_shop_id', 'cnt_user_id_item_brand_id', 'cnt_user_id_item_city_id']
(159028, 79)
((1094123, 67), (159028, 79))
(935095, 67)
(159028, 85)
((159028, 85), (159028, 85))

data 1
(1097520, 29)
encode cate id now
(1097520, 32)
fill nan now
(1097520, 32)
encode now
(1097520, 32)
decode time
(1097520, 34)
cate_match
(1097520, 40)
tongji2
(1097520, 53)
peng_feature
64.242880106
50.2037410736
48.9366660118
(1097520, 61)
(1097520, 67)
('get_last_time: ', 40.67762494087219)
['cnt_shop_id', 'cnt_item_id', 'cnt_user_id', 'cnt_item_brand_id', 'cnt_item_city_id', 'cnt_user_id_item_id'

In [14]:

print allday7.shape
allday7 = allday7.sample(frac=1.0)  # 全部打乱
test=allday7[allday7.model.notnull()].copy().reset_index().drop('index',axis=1)
print test.shape
allday7=allday7[(allday7.model.isnull())].copy().reset_index().drop('index',axis=1) 
print allday7.shape
cut_idx = int(round(0.2 * allday7.shape[0]))
val, train = allday7.iloc[:cut_idx], allday7.iloc[cut_idx:]
print val.shape,train.shape
del allday7

 

test.to_csv('/home/hou/Windy/IJCAI18/fusai/data/df_test.csv')
train.to_csv('/home/hou/Windy/IJCAI18/fusai/data/df_train.csv')
val.to_csv('/home/hou/Windy/IJCAI18/fusai/data/df_val.csv')

(1597063, 85)
(519888, 85)
(1077175, 85)
(215435, 85) (861740, 85)


In [17]:
# zhuan huan lv 
train=get_rate_feature_large(train,rate_saveall)
test=get_rate_feature_large(test,rate_saveall)
val=get_rate_feature_large(val,rate_saveall)

((10951924, 4), (861740, 85))
(9354861, 4)
(861740, 91)
((10951924, 4), (519888, 85))
(9354861, 4)
(519888, 91)
((10951924, 4), (215435, 85))
(9354861, 4)
(215435, 91)


In [3]:
test_save=pd.read_csv('/home/hou/Windy/IJCAI18/fusai/data/test.txt',sep=" ")
test=pd.read_csv('/home/hou/Windy/IJCAI18/fusai/data/df_test.csv')
train=pd.read_csv('/home/hou/Windy/IJCAI18/fusai/data/df_train.csv')
val=pd.read_csv('/home/hou/Windy/IJCAI18/fusai/data/df_val.csv')
test.shape,train.shape,val.shape

((519888, 80), (753840, 80), (323335, 80))

In [48]:
gc.collect()

308

In [26]:
def geshicuowu(data):
    for name in['item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
                'user_age_level', 'user_star_level', 'shop_review_num_level', 'shop_star_level', 'cate_match', 
                'context_page_id', 'hour', 'cnt_user_item_tmp', 'user_shop_cumcount', 'user_item_cumcount', 
                'shop_item_nunique', 'user_gender_id', 'user_occupation_id', 'cate_id', 'day', 'user_count_20',
                'user_shop_count_20', 'user_item_count_20', 'u_i_d_last_time', 'user_item_day_num', 'user_cate_day_num',
                'u_c_d_last_time']:
        data[name]=data[name].astype(float)
    print data.shape
    return data

In [27]:
train = geshicuowu(train)
test = geshicuowu(test)
val=geshicuowu(val)

In [39]:

test_save=pd.read_csv('/home/hou/Windy/IJCAI18/fusai/data/test.txt',sep=" ")
test_save['model']=test_save['user_id'].apply(lambda x:x%10)
test_save['predicted_score']=0

gc.collect()
rate_feature=['l_user_trade_all','l_user_trade_yes','l_item_trade_all','l_item_trade_yes']
count_list=['cnt_shop_id', 'cnt_item_id', 'cnt_user_id', 'cnt_item_brand_id', 'cnt_item_city_id', 'cnt_user_id_item_id', 'cnt_user_id_shop_id', 'cnt_user_id_item_brand_id', 'cnt_user_id_item_city_id']
last_feature=['u_i_d_last_time','user_item_day_num','user_cate_day_num','u_c_d_last_time']
peng_feature=['user_count_20','user_shop_count_20','user_item_count_20']
cate_feature=['user_gender_id','user_occupation_id','cate_id','day']
features=[ 'item_price_level', 'item_sales_level','item_collected_level','item_pv_level','user_age_level', 'user_star_level',
         'shop_review_num_level', 'shop_star_level','shop_review_positive_rate', 'shop_score_service','shop_score_delivery', 
        'shop_score_description','cate_match','context_page_id', 'hour']+['item_user_ratio','item_user_ratio2','cnt_user_item_tmp',
                  'pre_cate_price_rank','pre_cate_sale_rank','pre_cate_collect_rank','user_shop_cumcount',
         'user_item_cumcount','shop_item_nunique','user_price_rank','user_sale_rank','user_collect_rank'                                                        
         ]+\
        cate_feature+count_list+peng_feature+last_feature
target='is_trade'
print len(features)


gbm = lgb.LGBMClassifier(objective='binary',
                            max_depth=5,
                            num_leaves=16,
                            learning_rate=0.02,
                            n_estimators=20000,
                            colsample_bytree=0.3,#0.3
                            subsample = 0.75,#0.75
                            n_jobs=4,
                            lambda_l2=10,
                            seed=2018
                            )


gbm.fit(train[features],train[target],sample_weight=None,categorical_feature=cate_feature,
                eval_set=[(val[features],val[target])],eval_sample_weight=None,
                eval_metric='binary_logloss',
                early_stopping_rounds=40,verbose=20)

val_error=gbm.best_score_['valid_0']['binary_logloss']
print val_error
best_model=gbm.booster_
test['pred']=best_model.predict(test[features], num_iteration=best_model.best_iteration)
test_save=test_save.merge(test[['instance_id','pred']],on='instance_id',how='left')
# filter_t=test_save.model==i
test_save.loc[:,'predicted_score']=test_save.loc[:,'pred'].values
train_pred=best_model.predict(train[features], num_iteration=best_model.best_iteration)
print log_loss(train[target],train_pred)
gc.collect()
#test_save[['instance_id','predicted_score']].to_csv('baseline_y456_1.txt', index=False, sep=' ')

47
Training until validation scores don't improve for 40 rounds.
[20]	valid_0's binary_logloss: 0.457634
[40]	valid_0's binary_logloss: 0.335256
[60]	valid_0's binary_logloss: 0.266963
[80]	valid_0's binary_logloss: 0.227763
[100]	valid_0's binary_logloss: 0.205098
[120]	valid_0's binary_logloss: 0.192169
[140]	valid_0's binary_logloss: 0.184624
[160]	valid_0's binary_logloss: 0.180176
[180]	valid_0's binary_logloss: 0.177456
[200]	valid_0's binary_logloss: 0.175797
[220]	valid_0's binary_logloss: 0.17466
[240]	valid_0's binary_logloss: 0.173884
[260]	valid_0's binary_logloss: 0.173266
[280]	valid_0's binary_logloss: 0.172785
[300]	valid_0's binary_logloss: 0.172466
[320]	valid_0's binary_logloss: 0.172166
[340]	valid_0's binary_logloss: 0.171915
[360]	valid_0's binary_logloss: 0.171718
[380]	valid_0's binary_logloss: 0.171541
[400]	valid_0's binary_logloss: 0.171397
[420]	valid_0's binary_logloss: 0.17127
[440]	valid_0's binary_logloss: 0.171151
[460]	valid_0's binary_logloss: 0.17104

278

In [37]:

test_save=pd.read_csv('/home/hou/Windy/IJCAI18/fusai/data/test.txt',sep=" ")
test_save['model']=test_save['user_id'].apply(lambda x:x%10)
test_save['predicted_score']=0

test['pred']=best_model.predict(test[features], num_iteration=best_model.best_iteration)
test_save=test_save.merge(test[['instance_id','pred']],on='instance_id',how='left')
# filter_t=test_save.model==i
test_save.loc[:,'predicted_score']=test_save.loc[:,'pred'].values

gc.collect()
test_save[['instance_id','predicted_score']].to_csv('baseline_y456.txt', index=False, sep=' ')